In [ ]:
from os import path, mkdir

import numpy as np
from tqdm.notebook import tqdm
from itertools import product

from efsprapy.mcs1 import MCS1
from sfeprapy.mcs import InputParser
from fsetools.lib.fse_thermal_radiation_v2 import phi_solver
from sfeprapy.func.xlsx import dict_to_xlsx
import os

In [ ]:
dir_work = path.join(os.getcwd(), 'br187_base_case_analysis')
try:
    mkdir(dir_work)
except:
    pass

Wv, Hv = np.meshgrid(
    np.arange(3., 130 + 3., 3.),
    np.arange(3., 30. + 3., 3.)
)
Sv = np.zeros_like(Wv)

In [ ]:
for i, j in product(range(Wv.shape[0]), range(Wv.shape[1])):
        _, _, Sv[i, j], _ = phi_solver(
            W=Wv[i, j], H=Hv[i, j], w=Wv[i, j] / 2, h=Hv[i, j] / 2, theta=0, Q=84, Q_a=12.6, UA=1)

np.savetxt(
    path.join(dir_work, 'cases_with_s.csv'),
    np.column_stack((Wv.flatten(), Hv.flatten(), Sv.flatten())),
    delimiter=',', header='W,H,S', comments='', fmt='%.3f'
)

In [ ]:

def make_mcs_kwargs(W: float, H: float, S: float):
    return dict(
        n_simulations=100000,
        t_end=180. * 60.,
        t_step=10.,

        opening_width=W,
        opening_height=H,
        opening_ventilation_fraction=dict(dist='lognorm_mod_', lbound=1e-4, ubound=1 - 1e-4, mean=0.2, sd=0.2),

        room_width=W,
        room_width_depth_ratio=dict(dist='uniform_', lbound=0.4, ubound=0.6),
        room_height=H,

        # fire_mode=dict(dist='discrete_', values='0,1,2', weights='0.07,0.59,0.34'),
        fire_mode=0,
        fire_fuel_density_MJm2=dict(dist="br187_fuel_load_density_", lbound=10, ubound=5000),
        fire_combustion_efficiency=dict(dist='uniform_', lbound=0.8, ubound=1.0),
        fire_hrr_density_kWm2=dict(dist='br187_hrr_density_'),
        fire_growth_factor=0.0117,
        fire_t_lim=20. * 60.,
        fire_convection_factor=0.7,

        detector_to_fire_vertical_distance=H - 0.02,
        detector_to_fire_horizontal_distance=2.83,
        detector_act_temp=93 + 273.15,
        detector_response_time_index=250,
        detector_conduction_factor=0.65,

        lining_rho=2000,
        lining_c=1000,
        lining_k=1.13,

        receiver_ignition_temperature=-(300 + 273.15),
        receiver_emissivity=1,
        receiver_separation=S,

        ftp_chf=13.4e3,
        ftp_index=2.0,
        ftp_target=34592,
    )


In [ ]:

Iv = np.zeros_like(Wv, dtype=int)
for i in tqdm(range(Wv.shape[0]), location=0):
    kwargs = dict()
    for j in range(Wv.shape[1]):
        W, H, S = Wv[i, j], Hv[i, j], Sv[i, j]
        case_name = f'{W:03g}-{H:03g}-{S:06.3f}'
        kwargs[case_name] = make_mcs_kwargs(W=W, H=H, S=S).copy()

    fp_input = path.join(dir_work, f'br187_base_cases_w{i:03f}.xlsx')
    dict_to_xlsx({i: InputParser.flatten_dict(v) for i, v in kwargs.items()}, fp_input)

    pbar = tqdm(total=sum([v['n_simulations'] for k, v in kwargs.items()]), )
    mcs = MCS1()
    mcs.set_inputs_file_path(fp_input)
    mcs.run(n_proc=2, set_progress=lambda x: pbar.update(1), save=True, save_archive=True, location=1)
    pbar.close()

    for j in range(Wv.shape[1]):
        W, H, S = Wv[i, j], Hv[i, j], Sv[i, j]
        case_name = f'{W:03g}-{H:03g}-{S:06.3f}'
        t_ig_ftp = mcs[case_name].output[:, 1]
        Iv[i, j] = int(sum(np.bitwise_and(t_ig_ftp != np.inf, t_ig_ftp != np.nan)))


In [ ]:
np.savetxt(
    path.join(dir_work, 's_and_i.csv'),
    np.column_stack((Wv.flatten(), Hv.flatten(), Sv.flatten(), Iv.flatten())),
    delimiter=',',
    header='W,H,S,I',
    comments='',
    fmt='%.3f'
)